In [1]:
#SCRIPT: CARGA DE DADOS (Excel para SQL) 

import pandas as pd
from sqlalchemy import create_engine
import sys 
import openpyxl # Garante que a biblioteca de ler Excel está importada

print("--- INICIANDO TAREFA 1: CARGA DO BANCO DE DADOS ---")

# 1. CONFIGURAÇÃO DA CONEXÃO 
db_user = 'root'
db_password = 'SUA_SENHA_AQUI''
db_host = 'localhost'
db_port = '3306'
db_name = 'farmaceutica_hypera'

try:
    engine_string = f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
    engine = create_engine(engine_string)
    print(f"Conexão com o banco '{db_name}' estabelecida com sucesso!")
except Exception as e:
    print(f"ERRO FATAL ao conectar ao banco de dados: {e}")
    print("Verifique sua senha, usuário ou se o serviço do MySQL está rodando.")
    sys.exit() # Para o script se não conseguir conectar


# 2. DEFINIÇÃO DO ARQUIVO E ABAS ---

arquivo_excel = 'dados_farmaceuticos_hypera.xlsx' 

# Nomes das abas
aba_clientes = 'clientes'
aba_produtos = 'produtos'
aba_reps = 'representantes'
aba_vendas = 'vendas'
aba_pagamentos = 'pagamentos'


# 3. FUNÇÃO PARA CARREGAR DADOS (Lê Excel, Insere no SQL)
def carregar_dados_excel(arquivo, nome_aba, nome_tabela, engine_conexao, colunas_data=None):
    try:
        print(f"\nIniciando leitura da aba '{nome_aba}'...")
        df = pd.read_excel(arquivo, sheet_name=nome_aba)
        
        print(f"Lido com sucesso. Iniciando carga da tabela SQL '{nome_tabela}'...")
        
        if colunas_data:
            for col in colunas_data:
                df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # USA O "df.to_sql()" PARA INSERIR OS DADOS NO BANCO
        df.to_sql(nome_tabela, con=engine_conexao, if_exists='append', index=False)
        
        print(f"SUCESSO: {len(df)} linhas inseridas na tabela '{nome_tabela}'.")
    
    except Exception as e:
        print(f"ERRO ao carregar tabela '{nome_tabela}' da aba '{nome_aba}': {e}")
        print("Verifique se o nome do arquivo Excel e o nome da aba estão corretos.")
        print("Verifique também se você rodou o script CREATE TABLE no Workbench.")

#4. EXECUÇÃO DAS CARGAS 
print("\n INICIANDO CARGA DAS 5 TABELAS ")
carregar_dados_excel(arquivo_excel, aba_clientes, 'clientes', engine)
carregar_dados_excel(arquivo_excel, aba_produtos, 'produtos', engine)
carregar_dados_excel(arquivo_excel, aba_reps, 'representantes', engine)
carregar_dados_excel(arquivo_excel, aba_vendas, 'vendas', engine, colunas_data=['data'])
carregar_dados_excel(arquivo_excel, aba_pagamentos, 'pagamentos', engine, 
                     colunas_data=['data_emissao', 'data_vencimento', 'data_pagamento'])

print("\n-TAREFA 1 FINALIZADA ")
print("O banco de dados foi populado. Agora você pode ir para o notebook de ANÁLISE.")

--- INICIANDO TAREFA 1: CARGA DO BANCO DE DADOS ---
Conexão com o banco 'farmaceutica_hypera' estabelecida com sucesso!

--- INICIANDO CARGA DAS 5 TABELAS ---

Iniciando leitura da aba 'clientes'...
Lido com sucesso. Iniciando carga da tabela SQL 'clientes'...
ERRO ao carregar tabela 'clientes' da aba 'clientes': (mysql.connector.errors.IntegrityError) 1062 (23000): Duplicate entry '1' for key 'clientes.PRIMARY'
[SQL: INSERT INTO clientes (cliente_id, nome_cliente, tipo_cliente, regiao, uf, canal) VALUES (%(cliente_id)s, %(nome_cliente)s, %(tipo_cliente)s, %(regiao)s, %(uf)s, %(canal)s)]
[parameters: [{'cliente_id': 1, 'nome_cliente': 'Hospital São Lucas', 'tipo_cliente': 'Hospital', 'regiao': 'Sul', 'uf': 'RS', 'canal': 'Varejo'}, {'cliente_id': 2, 'nome_cliente': 'Farmácia Preço Justo', 'tipo_cliente': 'Farmácia Independente', 'regiao': 'Centro-Oeste', 'uf': 'DF', 'canal': 'Hospitalar'}, {'cliente_id': 3, 'nome_cliente': 'Rede Bem+Saúde', 'tipo_cliente': 'Rede de Farmácias', 'regiao'